# Gradient Ascent Fooling
This notebook contains the code to generate fooling images for the alex-net-based trafficsign pytorchmodel. 

The code is mostly documented throughout the lines, for an deeper insight what's going on please consider reading the paper. 

In [ ]:
import os
import cv2
import numpy as np

import torch
from torch.optim import SGD
from torchvision import models
from torch.nn import functional

from model import AlexNet

from constants import IMG_SIZE
from misc_functions import preprocess_image, recreate_image

The Gradient Ascent Fooling is based on the idea, to "improve" a random image towards the target confidence, by iteratively changing it. 

To step away from brute-forcing, the alternation is tuned with gradient-ascent, based on the model's predictions. This is done in the following block mostly in the function "generate(self)"

In [2]:
class FoolingSampleGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent, breaks as soon as
        the target prediction confidence is captured
    """
    def __init__(self, model, target_class, minimum_confidence):
        self.model = model
        self.model.eval()
        self.target_class = target_class
        self.minimum_confidence = minimum_confidence
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (IMG_SIZE, IMG_SIZE, 3)))
        # Create the folder to export images if not exists
        if not os.path.exists('generated'):
            os.makedirs('generated')

    def generate(self):
        img_file_extension = '.png'
        for i in range(1, iterations+1):      
            # Preprocess randomly generated image
            self.processed_image = preprocess_image(self.created_image)        
            # Define optimizer for the image, strong learning rate
            optimizer = SGD([self.processed_image], lr=6)
            # Feed the image Forward through the net
            output = self.model(self.processed_image)
            # Get confidence from softmax
            target_confidence = functional.softmax(output)[0][self.target_class]
            
            if target_confidence > self.minimum_confidence:
                # Reading the raw image and pushing it through model to see the prediction
                # this is needed because the format of preprocessed image is float and when
                # it is written back to file it is converted to uint8, so there is a chance that
                # there are some losses while writing
                confirmation_image = cv2.imread('generated/ga_fooling_class_' +
                                                str(self.target_class) + img_file_extension, 1)
                # Preprocess image
                confirmation_processed_image = preprocess_image(confirmation_image)
                # Get prediction
                confirmation_output = self.model(confirmation_processed_image)
                # Get confidence
                softmax_confirmation = \
                    functional.softmax(confirmation_output)[0][self.target_class]#.data.numpy()[0]
                if softmax_confirmation > self.minimum_confidence:
                    print('Generated fooling image with', "{0:.2f}".format(softmax_confirmation),
                          'confidence at', str(i) + 'th iteration.')
                    break
                    
            # Target specific class
            class_loss = -output[0, self.target_class]  
            print('Iteration:', str(i), 'Target Confidence', "{0:.4f}".format(target_confidence))
            # Zero grads - refresh current gradient 
            self.model.zero_grad()
            # Backward propagation - tune the weights for the optimizer
            class_loss.backward()
            # Update image - with new weights
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image)   
            # Save image
            cv2.imwrite('generated/ga_fooling_class_' + str(self.target_class) + img_file_extension,
                        self.created_image)  
        return self.processed_image

In [3]:
# Path to model
model_file_path = 'model.pth'

# Set device to cuda, if available, else use cpu as device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    # Set map location to gpu, if cuda is available and load model
    state_dict = torch.load(model_file_path)
else:
    # Set map location to cpu, if cuda is not available and load model
    state_dict = torch.load(model_file_path, map_location='cpu')
# Initialize model from "model.py"
model = AlexNet()
# Initialize model and device (either cuda or cpu)
model = model.to(device)
# Load state dictionary from model
model.load_state_dict(state_dict)

In [36]:
if __name__ == '__main__':
    for i in range(43): # Generates for each of all 43 classes one image
        pretrained_model = model
        target_class = i
        min_confidence = 0.99
        iterations = 200

        cig = FoolingSampleGeneration(pretrained_model, target_class, min_confidence)
        cig.generate()

/var/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Iteration: 1 Target Confidence 0.0007
Iteration: 2 Target Confidence 0.0009
Iteration: 3 Target Confidence 0.0246
Iteration: 4 Target Confidence 0.0034
Iteration: 5 Target Confidence 0.0000
Iteration: 6 Target Confidence 0.0000
Iteration: 7 Target Confidence 0.0000
Iteration: 8 Target Confidence 0.0000
Iteration: 9 Target Confidence 0.0000
Iteration: 10 Target Confidence 0.0000
Iteration: 11 Target Confidence 0.0000
Iteration: 12 Target Confidence 0.0000
Iteration: 13 Target Confidence 0.0000
Iteration: 14 Target Confidence 0.0000
Iteration: 15 Target Confidence 0.0000
Iteration: 16 Target Confidence 0.0000
Iteration: 17 Target Confidence 0.0000
Iteration: 18 Target Confidence 0.0000
Iteration: 19 Target Confidence 0.0000
Iteration: 20 Target Confidence 0.0000
Iteration: 21 Target Confidence 0.0000
Iteration: 22 Target Confidence 0.0000
Iteration: 23 Target Confidence 0.0000
Iteration: 24 Target Confidence 0.0000
Iteration: 25 Target Confidence 0.0000
Iteration: 26 Target Confidence 0.

/var/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Generated fooling image with 1.00 confidence at 3th iteration.
Iteration: 1 Target Confidence 0.0005
Iteration: 2 Target Confidence 0.5006
Generated fooling image with 1.00 confidence at 3th iteration.
Iteration: 1 Target Confidence 0.0047
Iteration: 2 Target Confidence 0.0076
Iteration: 3 Target Confidence 0.0045
Iteration: 4 Target Confidence 0.0003
Iteration: 5 Target Confidence 0.8906
Generated fooling image with 1.00 confidence at 6th iteration.
Iteration: 1 Target Confidence 0.0001
Iteration: 2 Target Confidence 0.0005
Iteration: 3 Target Confidence 0.0045
Iteration: 4 Target Confidence 0.0072
Iteration: 5 Target Confidence 0.0925
Iteration: 6 Target Confidence 0.0624
Generated fooling image with 1.00 confidence at 7th iteration.
Iteration: 1 Target Confidence 0.0001
Iteration: 2 Target Confidence 0.0209
Iteration: 3 Target Confidence 0.0027
Iteration: 4 Target Confidence 0.0000
Iteration: 5 Target Confidence 0.0000
Iteration: 6 Target Confidence 0.0000
Iteration: 7 Target Confid

Iteration: 192 Target Confidence 0.0000
Iteration: 193 Target Confidence 0.0000
Iteration: 194 Target Confidence 0.0000
Iteration: 195 Target Confidence 0.0000
Iteration: 196 Target Confidence 0.0000
Iteration: 197 Target Confidence 0.0000
Iteration: 198 Target Confidence 0.0000
Iteration: 199 Target Confidence 0.0000
Iteration: 200 Target Confidence 0.0000
Iteration: 1 Target Confidence 0.0001
Iteration: 2 Target Confidence 0.0452
Iteration: 3 Target Confidence 0.2336
Iteration: 4 Target Confidence 0.8669
Generated fooling image with 1.00 confidence at 5th iteration.
Iteration: 1 Target Confidence 0.0000
Iteration: 2 Target Confidence 0.0001
Iteration: 3 Target Confidence 0.0026
Iteration: 4 Target Confidence 0.0056
Iteration: 5 Target Confidence 0.0034
Iteration: 6 Target Confidence 0.0059
Iteration: 7 Target Confidence 0.0074
Iteration: 8 Target Confidence 0.0196
Iteration: 9 Target Confidence 0.0209
Iteration: 10 Target Confidence 0.0322
Iteration: 11 Target Confidence 0.0818
Itera

...

Iteration: 121 Target Confidence 0.0000
Iteration: 122 Target Confidence 0.0000
Iteration: 123 Target Confidence 0.0000
Iteration: 124 Target Confidence 0.0000
Iteration: 125 Target Confidence 0.0000
Iteration: 126 Target Confidence 0.0000
Iteration: 127 Target Confidence 0.0000
Iteration: 128 Target Confidence 0.0000
Iteration: 129 Target Confidence 0.0000
Iteration: 130 Target Confidence 0.0000
Iteration: 131 Target Confidence 0.0000
Iteration: 132 Target Confidence 0.0000
Iteration: 133 Target Confidence 0.0000
Iteration: 134 Target Confidence 0.0000
Iteration: 135 Target Confidence 0.0000
Iteration: 136 Target Confidence 0.0000
Iteration: 137 Target Confidence 0.0000
Iteration: 138 Target Confidence 0.0000
Iteration: 139 Target Confidence 0.0000
Iteration: 140 Target Confidence 0.0000
Iteration: 141 Target Confidence 0.0000
Iteration: 142 Target Confidence 0.0000
Iteration: 143 Target Confidence 0.0000
Iteration: 144 Target Confidence 0.0000
Iteration: 145 Target Confidence 0.0000


Iteration: 125 Target Confidence 0.0000
Iteration: 126 Target Confidence 0.0000
Iteration: 127 Target Confidence 0.0000
Iteration: 128 Target Confidence 0.0000
Iteration: 129 Target Confidence 0.0000
Iteration: 130 Target Confidence 0.0000
Iteration: 131 Target Confidence 0.0000
Iteration: 132 Target Confidence 0.0000
Iteration: 133 Target Confidence 0.0000
Iteration: 134 Target Confidence 0.0000
Iteration: 135 Target Confidence 0.0000
Iteration: 136 Target Confidence 0.0000
Iteration: 137 Target Confidence 0.0000
Iteration: 138 Target Confidence 0.0000
Iteration: 139 Target Confidence 0.0000
Iteration: 140 Target Confidence 0.0000
Iteration: 141 Target Confidence 0.0000
Iteration: 142 Target Confidence 0.0000
Iteration: 143 Target Confidence 0.0000
Iteration: 144 Target Confidence 0.0000
Iteration: 145 Target Confidence 0.0000
Iteration: 146 Target Confidence 0.0000
Iteration: 147 Target Confidence 0.0000
Iteration: 148 Target Confidence 0.0000
Iteration: 149 Target Confidence 0.0000


Iteration: 133 Target Confidence 0.0000
Iteration: 134 Target Confidence 0.0000
Iteration: 135 Target Confidence 0.0000
Iteration: 136 Target Confidence 0.0000
Iteration: 137 Target Confidence 0.0000
Iteration: 138 Target Confidence 0.0000
Iteration: 139 Target Confidence 0.0000
Iteration: 140 Target Confidence 0.0000
Iteration: 141 Target Confidence 0.0000
Iteration: 142 Target Confidence 0.0000
Iteration: 143 Target Confidence 0.0000
Iteration: 144 Target Confidence 0.0000
Iteration: 145 Target Confidence 0.0000
Iteration: 146 Target Confidence 0.0000
Iteration: 147 Target Confidence 0.0000
Iteration: 148 Target Confidence 0.0000
Iteration: 149 Target Confidence 0.0000
Iteration: 150 Target Confidence 0.0000
Iteration: 151 Target Confidence 0.0000
Iteration: 152 Target Confidence 0.0000
Iteration: 153 Target Confidence 0.0000
Iteration: 154 Target Confidence 0.0000
Iteration: 155 Target Confidence 0.0000
Iteration: 156 Target Confidence 0.0000
Iteration: 157 Target Confidence 0.0000


Iteration: 126 Target Confidence 0.0000
Iteration: 127 Target Confidence 0.0000
Iteration: 128 Target Confidence 0.0000
Iteration: 129 Target Confidence 0.0000
Iteration: 130 Target Confidence 0.0000
Iteration: 131 Target Confidence 0.0000
Iteration: 132 Target Confidence 0.0000
Iteration: 133 Target Confidence 0.0000
Iteration: 134 Target Confidence 0.0000
Iteration: 135 Target Confidence 0.0000
Iteration: 136 Target Confidence 0.0000
Iteration: 137 Target Confidence 0.0000
Iteration: 138 Target Confidence 0.0000
Iteration: 139 Target Confidence 0.0000
Iteration: 140 Target Confidence 0.0000
Iteration: 141 Target Confidence 0.0000
Iteration: 142 Target Confidence 0.0000
Iteration: 143 Target Confidence 0.0000
Iteration: 144 Target Confidence 0.0000
Iteration: 145 Target Confidence 0.0000
Iteration: 146 Target Confidence 0.0000
Iteration: 147 Target Confidence 0.0000
Iteration: 148 Target Confidence 0.0000
Iteration: 149 Target Confidence 0.0000
Iteration: 150 Target Confidence 0.0000


Iteration: 99 Target Confidence 0.0000
Iteration: 100 Target Confidence 0.0000
Iteration: 101 Target Confidence 0.0000
Iteration: 102 Target Confidence 0.0000
Iteration: 103 Target Confidence 0.0000
Iteration: 104 Target Confidence 0.0000
Iteration: 105 Target Confidence 0.0000
Iteration: 106 Target Confidence 0.0000
Iteration: 107 Target Confidence 0.0000
Iteration: 108 Target Confidence 0.0000
Iteration: 109 Target Confidence 0.0000
Iteration: 110 Target Confidence 0.0000
Iteration: 111 Target Confidence 0.0000
Iteration: 112 Target Confidence 0.0000
Iteration: 113 Target Confidence 0.0000
Iteration: 114 Target Confidence 0.0000
Iteration: 115 Target Confidence 0.0000
Iteration: 116 Target Confidence 0.0000
Iteration: 117 Target Confidence 0.0000
Iteration: 118 Target Confidence 0.0000
Iteration: 119 Target Confidence 0.0000
Iteration: 120 Target Confidence 0.0000
Iteration: 121 Target Confidence 0.0000
Iteration: 122 Target Confidence 0.0000
Iteration: 123 Target Confidence 0.0000
I